In [1]:
from dataclasses import dataclass
import torch
import torch.nn as nn


In [2]:
@dataclass
class Config: 
    d_model:int
    d_vocab:int
    d_hidden:int
    d_head:int
    seed:int

class MLP(nn.Module):
            def __init__(self,config):
                super().__init__()
                self.fc1 = nn.Linear(config.d_model, config.d_hidden)
                self.fc2 =nn.Linear(config.d_hidden,config.d_hidden)
                self.fc3 = nn.Linear(config.d_hidden,config.d_model)

            def forward(self, x):
                x = self.relu(self.fc1(x))
                x = self.relu(self.fc2(x))
                x = self.fc3(x)
                return x

class AttentionHead(nn.Module):
    def __init__(self,config:Config):
         self.WQ = nn.Parameter(torch.rand(config.d_model,config.d_head)) #W_Q
         self.WK_T = nn.Parameter(torch.rand(config.d_head,config.d_model)) 
         self.WOV =nn.Parameter(torch.rand(config.d_model,config.d_model))

    def forward(self,x):
        nc = x.size(1)
        fullM = torch.full(size = (nc,nc),fill_value= float("-Inf"))
        maskM = torch.triu(fullM, diagonal=1)

        left_mult = x @ self.WQ
        right_mult = self.WK_T @ x
        activation_input = left_mult @ right_mult + maskM
        acitvation_output = torch.nn.functional.softmax(activation_input,dim=1)

        return acitvation_output @ x @ self.WOV


class Transformer(nn.Module):
    def __init__(self,config:Config):
        super().__init__()
        self.AH1 = AttentionHead(config)
        self.MLP = MLP(config)

    def forward(self,x):
        return x + self.AH1.forward(x) + self.MLP.forward()
    
